In [36]:
from re import sub, compile
from requests import get, exceptions
from pandas import DataFrame, Series, set_option, read_csv
import numpy as np
import seaborn as sns
from pylab import rcParams, savefig
import matplotlib
from gensim.utils import tokenize
from gensim.models import doc2vec, Word2Vec
from nltk import sent_tokenize
from itertools import chain
from nltk.tokenize import wordpunct_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import models, corpora
from pymorphy2 import MorphAnalyzer
from pickle import load
from collections import namedtuple
morph = MorphAnalyzer()

Загрузка корпуса сообщений

In [5]:
documents = load(open('pickle/pr.pickle', 'rb'))

Загрузка модели

In [12]:
model = Word2Vec.load('2ch_model')

In [8]:
keyword = 'программирование'

Загрузка тестового корпуса

In [72]:
set_option('display.max_rows', None)
df = read_csv('test_set.txt', sep= '\n', names = ['text'])
test_y = [line.split('\n')[0] for line in open('test_set_y.txt')]
df['y'] = Series(test_y, index = df.index)

In [73]:
df['y'] = df.y.astype(int)
df['y'] = df.y.astype(bool)
df

,text,y
0,ПХП умерло в 2013 году. Если свой сайт сделаю то проще будет работку найти.,True
1,"PHP это ковырялки легаси, плохая производительность и scaling. В бекэнде пока что только GO из нормальных языков.",True
2,"Не позорь go сообщество, чушь несешь.",True
3,"После установки git появились git cmd и git bash. Для чего они нужны, если команды git работают при наборе в cmd.exe. Чтобы git каждый раз к имени команды не прибавлять.",True
4,даров где можно скачать какие нибудь паки картинок по категориям.по штук 200-1000 для разных категорий.,False
5,животное-млекопитающее-котик-котомaльчик :3,False
6,Так и предполагается. Что именно требуется сделать.,False
7,Ромбовидное наследование.Не обращай внимания.,True
8,Пол байта это 4 или 7 бит.,True
9,"Сап анон, подскажи сайт с фрилансерами для лайтовых задач. Типа решения лаб и т.д.",True


Оценка на усреднённом векторе

In [74]:
result = []

for test_word in df.text:
    amount = 0
    similarity = 0
    for i in wordpunct_tokenize(test_word):
        sim = 0
        try:
            sim = model.similarity(morph.parse(i)[0].normal_form, keyword)
            amount += 1
        except KeyError:
            pass
        similarity += sim
    positive = False
    sim_am = 0
    try:
        sim_am = similarity/amount
    except ZeroDivisionError:
        pass
    if sim_am > 0.5:
        positive = True
    result.append(positive)

In [75]:
df['result_mean'] = Series(result, index = df.index)
df

,text,y,result_mean
0,ПХП умерло в 2013 году. Если свой сайт сделаю то проще будет работку найти.,True,False
1,"PHP это ковырялки легаси, плохая производительность и scaling. В бекэнде пока что только GO из нормальных языков.",True,True
2,"Не позорь go сообщество, чушь несешь.",True,False
3,"После установки git появились git cmd и git bash. Для чего они нужны, если команды git работают при наборе в cmd.exe. Чтобы git каждый раз к имени команды не прибавлять.",True,False
4,даров где можно скачать какие нибудь паки картинок по категориям.по штук 200-1000 для разных категорий.,False,False
5,животное-млекопитающее-котик-котомaльчик :3,False,False
6,Так и предполагается. Что именно требуется сделать.,False,False
7,Ромбовидное наследование.Не обращай внимания.,True,False
8,Пол байта это 4 или 7 бит.,True,False
9,"Сап анон, подскажи сайт с фрилансерами для лайтовых задач. Типа решения лаб и т.д.",True,False


Подготовка TF-IDF словаря

In [76]:
corpus = documents
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = vectorizer.idf_
weights_dict = dict(zip(vectorizer.get_feature_names(), idf))

Оценка на взвешенном векторе

In [80]:
result = []

for test_word in df.text:
    amount = 0
    similarity = 0
    for i in wordpunct_tokenize(test_word):
        sim = 0
        try:
            sim = model.similarity(morph.parse(i)[0].normal_form, keyword)
            amount += 1
            try:
                weight = round(weights_dict[i], 2)
                sim *= weight
            except:
                pass
        except KeyError:
            pass
        similarity += sim
    positive = False
    sim_am = 0
    try:
        sim_am = similarity/amount
    except ZeroDivisionError:
        pass
    if sim_am > 0.5:
        positive = True
    result.append(positive)

In [81]:
df['result_mean'] = Series(result, index = df.index)
df

,text,y,result_mean
0,ПХП умерло в 2013 году. Если свой сайт сделаю то проще будет работку найти.,True,True
1,"PHP это ковырялки легаси, плохая производительность и scaling. В бекэнде пока что только GO из нормальных языков.",True,True
2,"Не позорь go сообщество, чушь несешь.",True,True
3,"После установки git появились git cmd и git bash. Для чего они нужны, если команды git работают при наборе в cmd.exe. Чтобы git каждый раз к имени команды не прибавлять.",True,True
4,даров где можно скачать какие нибудь паки картинок по категориям.по штук 200-1000 для разных категорий.,False,False
5,животное-млекопитающее-котик-котомaльчик :3,False,True
6,Так и предполагается. Что именно требуется сделать.,False,False
7,Ромбовидное наследование.Не обращай внимания.,True,True
8,Пол байта это 4 или 7 бит.,True,True
9,"Сап анон, подскажи сайт с фрилансерами для лайтовых задач. Типа решения лаб и т.д.",True,True
